In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random
from typing import Optional, Dict, Sequence
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from copy import deepcopy
import copy
import logging
import json
import loralib as lora

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
from dataclasses import dataclass
from transformers import AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy

In [3]:
config = AutoConfig.from_pretrained('skt/ko-gpt-trinity-1.2B-v0.5')
config

GPT2Config {
  "_name_or_path": "skt/ko-gpt-trinity-1.2B-v0.5",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 8,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1920,
  "n_head": 16,
  "n_inner": 7680,
  "n_layer": 24,
  "n_positions": 1024,
  "pad_token_id": 8,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.28.0",
  "use_cache": true,
  "vocab_size": 51200
}

In [4]:
config = AutoConfig.from_pretrained('skt/ko-gpt-trinity-1.2B-v0.5', n_ctx=128, n_inner=240)
config

GPT2Config {
  "_name_or_path": "skt/ko-gpt-trinity-1.2B-v0.5",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 8,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 128,
  "n_embd": 1920,
  "n_head": 16,
  "n_inner": 240,
  "n_layer": 24,
  "n_positions": 1024,
  "pad_token_id": 8,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.28.0",
  "use_cache": true,
  "vocab_size": 51200
}

In [5]:
huggingface_model_path = 'skt/ko-gpt-trinity-1.2B-v0.5'

model = AutoModelForCausalLM.from_pretrained(huggingface_model_path, config=config, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained(
    huggingface_model_path, bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at skt/ko-gpt-trinity-1.2B-v0.5 and are newly initialized because the shapes did not match:
- transformer.h.0.mlp.c_fc.weight: found shape torch.Size([1920, 7680]) in the checkpoint and torch.Size([1920, 240]) in the model instantiated
- transformer.h.0.mlp.c_fc.bias: found shape torch.Size([7680]) in the checkpoint and torch.Size([240]) in the model instantiated
- transformer.h.0.mlp.c_proj.weight: found shape torch.Size([7680, 1920]) in the checkpoint and torch.Size([240, 1920]) in the model instantiated
- transformer.h.1.mlp.c_fc.weight: found shape torch.Size([1920, 7680]) in the checkpoint and torch.Size([1920, 240]) in the model instantiated
- transformer.h.1.mlp.c_fc.bias: found shape torch.Size([7680]) in the checkpoint and torch.Size([240]) in the model instantiated
- transformer.h.1.mlp.c_proj.weight: found shape torch.Size([7680, 1920]) in the checkpoint and torch.Size([240, 1920]) in the model in

In [5]:
class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [6]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [7]:
data_path_1_SFT = os.getenv('HOME')+'/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl' 

train_dataset = SFT_dataset(data_path_1_SFT=data_path_1_SFT, tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [8]:
training_args = TrainingArguments(
    output_dir=os.getenv('HOME')+'/aiffel/KoChatGPT/output_1_SFT',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

trainer.train()
model.save_pretrained(os.getenv('HOME')+'/aiffel/KoChatGPT/output_1_SFT')

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,7.671100
1000,6.695900
1500,6.232700
2000,5.911700
2500,5.746100
3000,5.644200
3500,5.424200
4000,5.354500
4500,5.386200
5000,5.223000


In [6]:
generator = pipeline('text-generation', model=os.getenv('HOME')+'/aiffel/KoChatGPT/output_1_SFT', tokenizer=tokenizer)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이기 때문에 실제 상황에 대한 답변을 드리기 어렵습니다. 하지만 일반적으로 해당 호텔이나 브랜드에서 문의하시는 것이 좋을 것 같습니다. 감사합니다. \n\n\n1. 만약 인터넷 검색: 부동산 웹사이트나 게임, 온라인 쇼핑몰 등에서 확인해보시는 것을 추천드립니다.

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에, 이 질문에 대한 정보를 알 수 없습니다. 하지만 제가 답변드릴 수 있는 정보가 있으시면 더 정확한 답변을 드릴 수 있을 것입니다. 감사합니다. \n\n\n이 있다면 알려주시면 정확한 답변을 제공해 드릴 수 있을 것 같습니다."

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에, 정확한 정보를 알 수 없습니다. 해당 정보는 해당 기관나 공식 웹사이트에서 확인하시는 것이 좋을 것 같습니다. 감사합니다. \n\n\n- 부동산 홈페이지나 차량 등록 및 전화에 문의해보시는 것이 가장 자세한 정보를 알려주시면 정확한 답변을

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 AI 어시스턴트이기 때문에 답변을 드리기 어렵습니다. 하지만 제가 도와드릴 수 있는 것이 좋을 것 같습니다.\n\n이 어떤지 자세히 물어보세요! \n\n일가요? 감사합니다!\n1. 친구에서 연락해보세요.\n5. 언제든지 말씀


In [2]:
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained, config=config, ignore_mismatched_sizes=True)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [5]:
huggingface_model_path = 'skt/ko-gpt-trinity-1.2B-v0.5'

model = AutoModelForCausalLM.from_pretrained(huggingface_model_path, config=config, ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained(
    huggingface_model_path, bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained=huggingface_model_path, lora_rank=0, tokenizer=tokenizer).cuda()

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at skt/ko-gpt-trinity-1.2B-v0.5 and are newly initialized because the shapes did not match:
- transformer.h.0.mlp.c_fc.weight: found shape torch.Size([1920, 7680]) in the checkpoint and torch.Size([1920, 240]) in the model instantiated
- transformer.h.0.mlp.c_fc.bias: found shape torch.Size([7680]) in the checkpoint and torch.Size([240]) in the model instantiated
- transformer.h.0.mlp.c_proj.weight: found shape torch.Size([7680, 1920]) in the checkpoint and torch.Size([240, 1920]) in the model instantiated
- transformer.h.1.mlp.c_fc.weight: found shape torch.Size([1920, 7680]) in the checkpoint and torch.Size([1920, 240]) in the model instantiated
- transformer.h.1.mlp.c_fc.bias: found shape torch.Size([7680]) in the checkpoint and torch.Size([240]) in the model instantiated
- transformer.h.1.mlp.c_proj.weight: found shape torch.Size([7680, 1920]) in the checkpoint and torch.Size([240, 1920]) in the model in

Some weights of the model checkpoint at skt/ko-gpt-trinity-1.2B-v0.5 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
data_path_2_RM = os.getenv('HOME')+'/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'

with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


In [7]:
random.seed(230319)
random.shuffle(total_data_ranking2chosen)

train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

100%|██████████| 200/200 [00:00<00:00, 1172.27it/s]


In [8]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=2,
                             max_epochs=1)

trainer.fit(use_lora=0)

model.save_pretrained(os.getenv('HOME')+'aiffel/KoChatGPT/output_2_RM')

Train step of epoch 0:   0%|          | 0/500 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 │   │   │   │   │   │   │    batch_size=2,                                                  │
│    7 │   │   │   │   │   │   │    max_epochs=1)                                                  │
│    8                                                                                             │
│ ❱  9 trainer.fit(use_lora=0)                                                                     │
│   10                                                                                             │
│   11 model.save_pretrained(os.getenv('HOME')+'aiffel/KoChatGPT/output_2_RM')                     │
│   12                                                                                             │
│                                                                                                  │
│ /aiffel/aiffel/aiffel/AIFFEL_quest_rs/GoingDeeper/Gd09/chatgpt/trainer/rm.py:66 in fit           │
│                                                                                                  │
│   63 │   │   │   │   reject_ids = reject_ids.squeeze(1).cuda()                                   │
│   64 │   │   │   │   r_mask = r_mask.squeeze(1).cuda()                                           │
│   65 │   │   │   │   chosen_reward = self.model(chosen_ids, attention_mask=c_mask)               │
│ ❱ 66 │   │   │   │   reject_reward = self.model(reject_ids, attention_mask=r_mask)               │
│   67 │   │   │   │   loss = self.loss_fn(chosen_reward, reject_reward)                           │
│   68 │   │   │   │   self.strategy.backward(loss, self.model, self.optimizer)                    │
│   69 │   │   │   │   self.strategy.optimizer_step(self.optimizer)                                │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/torch/nn/modules/module.py:1130 in _call_impl             │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1131 │   │   # Do not call functions when jit is used                                          │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /aiffel/aiffel/aiffel/AIFFEL_quest_rs/GoingDeeper/Gd09/chatgpt/models/base/reward_model.py:37 in │
│ forward                                                                                          │
│                                                                                                  │
│   34 │   │   │   self.value_head = nn.Linear(model.config.n_embd, 1)                             │
│   35 │                                                                                           │
│   36 │   def forward(self, sequences: torch.LongTensor, attention_mask: Optional[torch.Tensor    │
│ ❱ 37 │   │   outputs = self.model(sequences, attention_mask=attention_mask)                      │
│   38 │   │   last_hidden_states = outputs['last_hidden_state']                                   │
│   39 │   │   values = self.value_head(last_hidden_states)[:

In [ ]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

In [ ]:
input_text = "인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다. AI는 현대적인 컴퓨팅 혁신에서 중추적인 역할을 하며 개인과 비즈니스의 가치를 창출합니다. 예를 들어 광학 문자 인식(OCR)은 AI를 사용해 이미지 및 문서에서 텍스트 및 데이터를 추출하고, 구조화되지 않은 콘텐츠를 비즈니스에 바로 사용할 수 있게 만들고, 유용한 정보를 창출합니다."

output_reward = inference_RM(input_text=input_text)

In [ ]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained=os.getenv('HOME')+'/aiffel/KoChatGPT/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained=os.getenv('HOME')+'aiffel/KoChatGPT/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [ ]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [ ]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

In [ ]:
with open(os.getenv('HOME')+'/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [ ]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=2, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

model.save_pretrained(os.getenv('HOME')+'aiffel/KoChatGPT/output_3_PPO')

In [ ]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)